<a href="https://colab.research.google.com/github/ben45123/AWS-Lambda-Research/blob/main/rag_news_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Cell 1: Install the updated packages
!pip install pinecone-client==6.0.2 langchain langchain_openai openai langgraph pydantic langchain_pinecone

ERROR: Ignored the following yanked versions: 2.2.0, 2.2.5
ERROR: Could not find a version that satisfies the requirement pinecone-client==6.0.2 (from versions: 0.7.5, 0.7.11, 0.7.13, 0.7.16, 0.7.17, 0.7.26, 0.7.27, 0.7.42, 0.7.44, 0.7.46, 0.8.1, 0.8.5, 0.8.7, 0.8.8, 0.8.9, 0.8.10, 0.8.27, 0.8.33, 0.8.34, 0.8.36, 0.8.37, 0.8.38, 0.8.39b0, 0.8.39rc0, 0.8.39, 0.8.53, 0.8.56, 0.8.58, 0.8.59, 0.8.60, 2.0.0, 2.0.1, 2.0.2, 2.0.3, 2.0.4, 2.0.5, 2.0.6, 2.0.7, 2.0.8, 2.0.9, 2.0.10, 2.0.11, 2.0.12, 2.0.13, 2.1.0, 2.2.1, 2.2.2rc1, 2.2.2rc3, 2.2.2rc4, 2.2.2, 2.2.3rc1, 2.2.3, 2.2.4, 2.2.5rc2, 2.3.0.dev1, 3.0.0.dev1, 3.0.0.dev2, 3.0.0.dev3, 3.0.0.dev4, 3.0.0.dev5, 3.0.0.dev6, 3.0.0.dev7, 3.0.0.dev8, 3.0.0.dev9, 3.0.0.dev10, 3.0.0rc2, 3.0.0rc3, 3.0.0, 3.0.1, 3.0.2, 3.0.3, 3.1.0.dev1, 3.1.0, 3.2.0, 3.2.1, 3.2.2, 4.0.0, 4.1.0, 4.1.1, 4.1.2, 5.0.0, 5.0.1, 6.0.0.dev3, 6.0.0)
ERROR: No matching distribution found for pinecone-client==6.0.2


In [2]:
# Cell 2: Import libraries and set API keys
import os
import json
import pandas as pd
from typing import List, Dict, Any
import pinecone
from google.colab import userdata

# Set API keys from Colab userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')
PINECONE_API_KEY = userdata.get('PINECONE_API_KEY')  # Or use your provided key if saved
# If you're using the key from your provided code
# PINECONE_API_KEY = "pcsk_6akU8Z_2BXXXDSBKbvFCn4sciNM2FeJC6PwAt6wFwQeQjoJKDSjysRbtyBAdUfRv6z87e6"

# Set environment variables (some LangChain components use these)
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

# Configuration for Pinecone
PINECONE_INDEX_NAME = "cus635"
NAMESPACE = "Team_1"
CATEGORY = "Finance"

print("API keys loaded successfully!")


API keys loaded successfully!


In [3]:
# Cell 3: Import LangChain and LangGraph components
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
# Use the correct import for Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.chains import ConversationalRetrievalChain
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.messages import AIMessage, HumanMessage
from langgraph.graph import StateGraph, END
from pydantic import BaseModel, Field

# Print version for debugging
import pinecone
print(f"Pinecone SDK version: {pinecone.__version__}")

# Initialize models
embeddings = OpenAIEmbeddings()
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0.2
)

print("LangChain components initialized!")

Pinecone SDK version: 6.0.2
LangChain components initialized!


In [4]:
# Cell 4: Create the NewsRAG class for basic RAG functionality
class NewsRAG:
    """
    A RAG system that uses LangChain to answer questions based on news articles
    stored in Pinecone.
    """

    def __init__(self, api_key: str, index_name: str, namespace: str, category: str):
        """
        Initialize the NewsRAG with Pinecone credentials and team information.

        Args:
            api_key: Pinecone API key
            index_name: Pinecone index name
            namespace: Namespace (team name)
            category: News category
        """
        self.api_key = api_key
        self.index_name = index_name
        self.namespace = namespace
        self.category = category
        self.pinecone_index = None
        self.retriever = None
        self.rag_chain = None
        self.conversational_memory = []

    def connect_to_pinecone(self):
        """Connect to Pinecone and initialize the index."""
        try:
            # Initialize Pinecone
            pinecone_client = pinecone.Pinecone(api_key=self.api_key)
            self.pinecone_index = pinecone_client.Index(self.index_name)
            print(f"Connected to Pinecone index: {self.index_name}")
            return True
        except Exception as e:
            print(f"Error connecting to Pinecone: {str(e)}")
            return False

    def initialize_retriever(self):
        """Initialize the retriever from the Pinecone index."""
        try:
            # Make sure we have the Pinecone API key in env vars
            os.environ["PINECONE_API_KEY"] = self.api_key

            # Create a vector store using the updated PineconeVectorStore class
            vectorstore = PineconeVectorStore(
                embedding=embeddings,
                index_name=self.index_name,
                namespace=self.namespace,
                text_key="text"
            )

            # Create the retriever with filters
            self.retriever = vectorstore.as_retriever(
                search_kwargs={
                    "k": 5,
                    "filter": {"category": self.category}
                }
            )

            print(f"Retriever initialized for category: {self.category}")
            return True
        except Exception as e:
            print(f"Error initializing retriever: {str(e)}")
            return False

    def initialize_rag_chain(self):
        """Initialize the basic RAG chain."""
        # Define the prompt template
        template = """You are an AI assistant specialized in analyzing news articles in the {category} domain.
        Use the following retrieved news articles to answer the question.

        Retrieved articles:
        {context}

        Question: {question}

        Answer the question based on the retrieved articles. If the retrieved articles don't contain the information
        needed to answer the question accurately, acknowledge the limitations and provide the best answer possible
        based on available information. Include relevant sources in your response.
        """

        # Create the prompt
        prompt = ChatPromptTemplate.from_template(template)

        # Create the RAG chain
        self.rag_chain = (
            {"context": self.retriever, "question": RunnablePassthrough(), "category": lambda _: self.category}
            | prompt
            | llm
            | StrOutputParser()
        )

        print("Basic RAG chain initialized")
        return True

    def query(self, question: str) -> str:
        """
        Query the RAG system with a question.

        Args:
            question: User's question

        Returns:
            Answer based on the retrieved documents
        """
        if not self.rag_chain:
            self.initialize_rag_chain()

        try:
            return self.rag_chain.invoke(question)
        except Exception as e:
            print(f"Error querying RAG: {str(e)}")
            return f"Error processing your query: {str(e)}"

In [5]:
# Cell 5: Add conversational RAG functionality
class ConversationalNewsRAG(NewsRAG):
    """Extends NewsRAG with conversational capabilities."""

    def __init__(self, api_key: str, index_name: str, namespace: str, category: str):
        """Initialize using parent constructor."""
        super().__init__(api_key, index_name, namespace, category)
        self.chat_history = []
        self.conversational_rag = None

    def initialize_conversational_rag(self):
        """Initialize a conversational RAG chain with memory."""
        if not self.retriever:
            self.initialize_retriever()

        self.conversational_rag = ConversationalRetrievalChain.from_llm(
            llm=llm,
            retriever=self.retriever,
            return_source_documents=True
        )

        print("Conversational RAG chain initialized")
        return True

    def conversational_query(self, question: str) -> Dict:
        """
        Query the conversational RAG system with chat history.

        Args:
            question: User's question

        Returns:
            Answer and source documents
        """
        if not self.conversational_rag:
            self.initialize_conversational_rag()

        try:
            # Get response using chat history
            result = self.conversational_rag.invoke({
                "question": question,
                "chat_history": self.chat_history
            })

            # Update chat history
            self.chat_history.append((question, result["answer"]))

            return result
        except Exception as e:
            print(f"Error in conversational query: {str(e)}")
            return {"answer": f"Error processing your query: {str(e)}", "source_documents": []}

In [6]:
# Cell 6: Debugging Cell - Run this first to validate connections
import os
import pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_openai import OpenAIEmbeddings

# Print version information
print(f"Pinecone SDK version: {pinecone.__version__}")

# Initialize Pinecone
try:
    pinecone_client = pinecone.Pinecone(api_key=PINECONE_API_KEY)
    print("Pinecone client initialized successfully")

    # List available indexes
    indexes = pinecone_client.list_indexes()
    print(f"Available indexes: {indexes}")

    # Connect to the specific index
    index = pinecone_client.Index(PINECONE_INDEX_NAME)
    print(f"Connected to index: {PINECONE_INDEX_NAME}")

    # Verify index stats
    stats = index.describe_index_stats()
    print(f"Index stats: {stats}")

    # Try initializing the vector store
    try:
        embeddings = OpenAIEmbeddings()
        vectorstore = PineconeVectorStore(
            embedding=embeddings,
            index_name=PINECONE_INDEX_NAME,
            namespace=NAMESPACE,
            text_key="text"
        )
        print("Vector store initialized successfully!")

        # Try creating a retriever
        retriever = vectorstore.as_retriever(
            search_kwargs={
                "k": 5,
                "filter": {"category": CATEGORY}
            }
        )
        print("Retriever created successfully!")

    except Exception as e:
        print(f"Error initializing vector store: {str(e)}")

except Exception as e:
    print(f"Error connecting to Pinecone: {str(e)}")

# Test with a basic connection
print("\nTesting with a simple query...")
try:
    # Create a test embedding
    test_embedding = embeddings.embed_query("Test query about finance")
    # Try a simple query to the index
    query_response = index.query(
        vector=test_embedding,
        top_k=1,
        namespace=NAMESPACE,
        include_metadata=True
    )
    print("Query successful!")
    print(f"First result metadata: {query_response.matches[0].metadata if query_response.matches else 'No matches'}")
except Exception as e:
    print(f"Error querying index: {str(e)}")

Pinecone SDK version: 6.0.2
Pinecone client initialized successfully
Available indexes: [{
    "name": "cus635",
    "metric": "cosine",
    "host": "cus635-g311jqa.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1024,
    "deletion_protection": "disabled",
    "tags": null,
    "embed": {
        "model": "llama-text-embed-v2",
        "field_map": {
            "text": "text"
        },
        "dimension": 1024,
        "metric": "cosine",
        "write_parameters": {
            "dimension": 1024.0,
            "input_type": "passage",
            "truncate": "END"
        },
        "read_parameters": {
            "dimension": 1024.0,
            "input_type": "query",
            "truncate": "END"
        },
        "vector_type": "dense"
    }
}, {
    "name": "c